In [7]:
!pip install albumentations --user

  Using cached albumentations-2.0.6-py3-none-any.whl.metadata (43 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached albumentations-2.0.6-py3-none-any.whl (332 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)


In [6]:
import os
import cv2
import albumentations as A
from tqdm.notebook import tqdm
import  gc

In [18]:


# === PATHS ===
BASE_PATH = "dataset/train"
image_dir = os.path.join(BASE_PATH, "images")
label_dir = os.path.join(BASE_PATH, "labels")
aug_image_dir = os.path.join(BASE_PATH, "images_aug")
aug_label_dir = os.path.join(BASE_PATH, "labels_aug")

# Create output directories
os.makedirs(aug_image_dir, exist_ok=True)
os.makedirs(aug_label_dir, exist_ok=True)

# === AUGMENTATION PIPELINE ===
transform = A.Compose([
    A.Resize(640, 640, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], check_each_transform=False))

# === UTILITY FUNCTIONS ===

def read_label(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    boxes, labels = [], []
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue  # skip malformed lines
        labels.append(int(parts[0]))
        boxes.append([float(x) for x in parts[1:]])
    return boxes, labels

def write_label(file_path, labels, boxes):
    with open(file_path, 'w') as f:
        for label, box in zip(labels, boxes):
            f.write(f"{label} {' '.join(map(lambda x: str(round(x, 6)), box))}\n")

# === MAIN LOOP ===

files = sorted(os.listdir(image_dir))[:]

for filename in tqdm(files):
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    image_path = os.path.join(image_dir, filename)
    label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")

    if not os.path.exists(label_path):
        print(f"Missing label for {filename}, creating empty label.")
        open(label_path, 'a').close()  # create empty label

    image = cv2.imread(image_path)
    if image is None:
        print(f"Unreadable image: {filename}, skipping.")
        continue

    boxes, labels = read_label(label_path)

    # Apply transformation 
    try:
        transformed = transform(image=image, bboxes=boxes, class_labels=labels)
    except Exception as e:
        print(f"Error during transform on {filename}: {e}")
        continue

    aug_img = transformed['image']
    aug_boxes = transformed['bboxes']
    aug_labels = transformed['class_labels']

    # Clip bounding boxes to [0, 1]
    aug_boxes = [[min(max(coord, 0.0), 1.0) for coord in box] for box in aug_boxes]

    # Save augmented image
    new_img_name = f"{os.path.splitext(filename)[0]}_aug.jpg"
    cv2.imwrite(os.path.join(aug_image_dir, new_img_name), aug_img)

    # Save label 
    new_lbl_name = new_img_name.replace(".jpg", ".txt")
    write_label(os.path.join(aug_label_dir, new_lbl_name), aug_labels, aug_boxes)

    # Memory cleanup
    del image, aug_img
    gc.collect()


  0%|          | 0/14122 [00:00<?, ?it/s]

Error during transform on AoF04522.jpg: Expected x_max for bbox [0.9568835  0.22764227 1.0007564  0.40487805 1.        ] to be in the range [0.0, 1.0], got 1.0007563829421997.
Error during transform on AoF04568.jpg: Expected x_max for bbox [0.95510834 0.2234957  1.000774   0.265043   1.        ] to be in the range [0.0, 1.0], got 1.0007740259170532.
Error during transform on AoF04627.jpg: Expected x_min for bbox [-0.0015625   0.2888889   0.053125    0.39444444  1.        ] to be in the range [0.0, 1.0], got -0.0015625003725290298.
Error during transform on AoF04663.jpg: Expected x_min for bbox [-7.8125298e-04  2.7638888e-01  6.6406250e-02  4.0833336e-01
  1.0000000e+00] to be in the range [0.0, 1.0], got -0.0007812529802322388.
Error during transform on AoF04664.jpg: Expected x_min for bbox [-0.0015625   0.28333333  0.06796875  0.4         1.        ] to be in the range [0.0, 1.0], got -0.0015624985098838806.
Error during transform on AoF04687.jpg: Expected x_min for bbox [-7.8124925e-

In [20]:
import shutil
import os
from tqdm import tqdm

# Merge augmented images into original image folder
for f in tqdm(os.listdir(aug_image_dir), desc="Merging images"):
    src = os.path.join(aug_image_dir, f)
    dst = os.path.join(image_dir, f)
    if not os.path.exists(dst):  # Avoid overwriting
        shutil.copy(src, dst)

# Merge augmented labels into original label folder
for f in tqdm(os.listdir(aug_label_dir), desc="Merging labels"):
    src = os.path.join(aug_label_dir, f)
    dst = os.path.join(label_dir, f)
    if not os.path.exists(dst):  # Avoid overwriting
        shutil.copy(src, dst)


Merging labels: 100%|███████████████████████████████████████████████████████████| 13898/13898 [01:02<00:00, 221.53it/s]
